In [21]:
# БЛОК 1: ИМПОРТ НЕОБХОДИМЫХ БИБЛИОТЕК

import numpy as np # numpy - основная библиотека для работы с числовыми массивами и математическими операциями
import struct # struct - для работы с бинарными данными, необходим при чтении файлов формата IDX
from urllib.request import urlretrieve # urlretrieve - для скачивания файлов по URL
import gzip # gzip - для работы с сжатыми gzip файлами
import os # os - для проверки существования файлов в файловой системе

In [22]:
# БЛОК 2: ФУНКЦИИ ДЛЯ ЗАГРУЗКИ ДАННЫХ

def download_mnist_alt():
    """
    Функция для скачивания архивированных файлов датасета MNIST с альтернативного источника
    """
    # Базовый URL-адрес зеркала, где хранятся файлы датасета
    base_url = 'https://ossci-datasets.s3.amazonaws.com/mnist/'
    
    # Список файлов, которые необходимо скачать
    files = ['train-images-idx3-ubyte.gz',  # Изображения для обучения
             'train-labels-idx1-ubyte.gz',  # Метки для обучения
             't10k-images-idx3-ubyte.gz',   # Изображения для тестирования (10k = 10000)
             't10k-labels-idx1-ubyte.gz']   # Метки для тестирования

    # Проходим по всем файлам в списке
    for file in files:
        # Проверяем, существует ли файл в текущей директории
        if not os.path.exists(file):
            print(f"Скачивается: {file}") # Если файл не существует, выводим сообщение о начале загрузки
            urlretrieve(base_url + file, file) # Скачиваем файл: urlretrieve(URL_файла, имя_локального_файла)
            print(f"Загружен: {file}") # Выводим сообщение об успешной загрузке
        else:
            print(f"Файл уже существует: {file}") # Если файл уже существует, сообщаем об этом


def load_mnist_images(filename, max_samples=10000):
    """
    Функция для загрузки и обработки изображений из файла формата IDX
    
    Параметры:
    filename - имя файла с изображениями
    max_samples - максимальное количество образцов для загрузки
    """
    # Открываем gzip-архив в режиме чтения бинарных данных
    with gzip.open(filename, 'rb') as f:
        # Читаем заголовок файла: 4 беззнаковых целых числа (magic, количество, строки, столбцы)
        # Формат '>IIII': 
        #   '>' - big-endian порядок байт (стандарт для IDX)
        #   'I' - unsigned int (4 байта)
        magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
        
        images = np.frombuffer(f.read(), dtype=np.uint8) # Читаем оставшиеся данные как одномерный массив байтов
        
        # Преобразуем одномерный массив в двумерный:
        #   - каждая строка представляет одно изображение
        #   - количество пикселей в изображении = rows * cols = 28*28 = 784
        #   - выбираем только первые max_samples изображений
        images = images.reshape(-1, rows * cols)[:max_samples]
    
    # Нормализуем значения пикселей из диапазона [0, 255] в диапазон [0.0, 1.0]
    # Деление на 255.0 преобразует целые числа в вещественные
    return images / 255.0


def load_mnist_labels(filename, max_samples=10000):
    """
    Функция для загрузки меток (цифр) из файла формата IDX
    
    Параметры:
    filename - имя файла с метками
    max_samples - максимальное количество меток для загрузки
    """
    # Открываем gzip-архив в режиме чтения бинарных данных
    with gzip.open(filename, 'rb') as f:
        magic, num = struct.unpack(">II", f.read(8)) # Читаем заголовок файла: 2 беззнаковых целых числа (magic, количество меток)
        
        # Читаем оставшиеся данные как одномерный массив байтов
        # Каждый байт представляет метку (цифру от 0 до 9)
        labels = np.frombuffer(f.read(), dtype=np.uint8)[:max_samples]
    
    return labels # Возвращаем массив меток

In [23]:
# БЛОК 3: ЗАГРУЗКА И ПОДГОТОВКА ДАННЫХ

print("Загрузка MNIST с альтернативного зеркала...") # Выводим сообщение о начале загрузки данных

# Вызываем функцию скачивания файлов
download_mnist_alt()  # Используем новую функцию

X_train = load_mnist_images('train-images-idx3-ubyte.gz', 10000) # Загружаем обучающие изображения (первых 10000 образцов)
y_train = load_mnist_labels('train-labels-idx1-ubyte.gz', 10000) # Загружаем соответствующие метки для обучающих изображений

X_test = load_mnist_images('t10k-images-idx3-ubyte.gz', 2000) # Загружаем тестовые изображения (первых 2000 образцов)
y_test = load_mnist_labels('t10k-labels-idx1-ubyte.gz', 2000) # Загружаем соответствующие метки для тестовых изображений

# Выводим информацию о размерах загруженных данных
# X_train.shape вернет кортеж (количество_образцов, количество_признаков)
print(f"Обучающая выборка: {X_train.shape}")
print(f"Тестовая выборка: {X_test.shape}")

Загрузка MNIST с альтернативного зеркала...
Файл уже существует: train-images-idx3-ubyte.gz
Файл уже существует: train-labels-idx1-ubyte.gz
Файл уже существует: t10k-images-idx3-ubyte.gz
Файл уже существует: t10k-labels-idx1-ubyte.gz
Обучающая выборка: (10000, 784)
Тестовая выборка: (2000, 784)


In [24]:
# БЛОК 4: ФУНКЦИИ АКТИВАЦИИ И ИХ ПРОИЗВОДНЫЕ

def sigmoid(x):
    """
    Сигмоидная (логистическая) функция активации
    Преобразует входные значения в диапазон (0, 1)
    Формула: σ(x) = 1 / (1 + e^(-x))
    """
    return 1 / (1 + np.exp(-x))


def sigmoid_derivative(x):
    """
    Производная сигмоидной функции
    Необходима для обратного распространения ошибки
    Формула: σ'(x) = σ(x) * (1 - σ(x))
    В данном случае x уже является выходом сигмоиды
    """
    return x * (1 - x)


def relu(x):
    """
    Функция активации ReLU (Rectified Linear Unit)
    Возвращает максимум из 0 и входного значения
    Формула: ReLU(x) = max(0, x)
    """
    return np.maximum(0, x)


def relu_derivative(x):
    """
    Производная функции ReLU
    Возвращает 1 для положительных значений и 0 для отрицательных
    (x > 0) создает булев массив, astype(float) преобразует его в числовой
    """
    return (x > 0).astype(float)


def softmax(x):
    """
    Функция активации Softmax
    Преобразует вектор чисел в вектор вероятностей (сумма = 1)
    Используется для многоклассовой классификации
    """
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True)) # Для численной стабильности вычитаем максимум из каждого элемента
    return exp_x / np.sum(exp_x, axis=1, keepdims=True) # Нормализуем, чтобы сумма по строкам была равна 1

In [25]:
# БЛОК 5: ОДНОСЛОЙНАЯ НЕЙРОННАЯ СЕТЬ

class SingleLayerNN:
    """
    Класс, реализующий однослойную нейронную сеть
    Соответствует пункту 4 задания: однослойная сеть с сигмоидой
    """
    
    def __init__(self, input_size, output_size):
        """
        Конструктор класса
        Инициализирует веса и смещения нейронной сети
        Параметры:
        input_size - количество входных признаков (для MNIST = 784)
        output_size - количество нейронов в выходном слое (для цифр = 10)
        """
        # Инициализация весов матрицей размера (input_size × output_size)
        # Умножение на 0.01 делает начальные веса небольшими
        self.weights = np.random.randn(input_size, output_size) * 0.01
       
        self.bias = np.zeros((1, output_size)) # Инициализация смещений нулевым вектором размера (1 × output_size)
    
    def forward(self, X):
        """
        Прямое распространение (forward propagation)
        Вычисляет выход сети для заданного входного вектора X
        Параметры:
        X - входная матрица размера (количество_примеров × input_size)
        """
        self.z = np.dot(X, self.weights) + self.bias # Линейная комбинация: z = X·W + b
        self.a = sigmoid(self.z) # Применение функции активации (сигмоида)
        
        return self.a # Возвращаем активации выходного слоя
    
    def backward(self, X, y, output, learning_rate):
        """
        Обратное распространение ошибки (backward propagation)
        Вычисляет градиенты и обновляет веса
        Параметры:
        X - входные данные
        y - истинные метки (цифры)
        output - предсказания сети
        learning_rate - скорость обучения
        """
        m = X.shape[0] # Количество примеров в пакете
        
        # Преобразуем метки в one-hot encoding
        y_one_hot = np.zeros((m, 10)) # Создаем матрицу нулей размером (m × 10)
        y_one_hot[np.arange(m), y] = 1 # Для каждого примера устанавливаем 1 в столбце, соответствующем метке
        error = output - y_one_hot # Вычисляем ошибку: разница между предсказанием и истинным значением
        
        # Вычисляем градиенты:
        # dW = (1/m) * X^T · error
        dW = np.dot(X.T, error) / m
        # db = (1/m) * sum(error)
        db = np.sum(error, axis=0, keepdims=True) / m
        
        # Обновляем веса и смещения:
        # W = W - learning_rate * dW
        self.weights -= learning_rate * dW
        # b = b - learning_rate * db
        self.bias -= learning_rate * db
    
    def predict(self, X):
        """
        Функция предсказания
        Возвращает предсказанные классы (цифры) для входных данных X
        """
        output = self.forward(X) # Выполняем прямое распространение
        return np.argmax(output, axis=1) # Выбираем нейрон с максимальной активацией (argmax по строкам)
    
    def train(self, X, y, epochs=100, learning_rate=0.1):
        """
        Функция обучения сети
        Параметры:
        X - входные данные для обучения
        y - метки для обучения
        epochs - количество эпох обучения
        learning_rate - скорость обучения
        """
        # Цикл по эпохам обучения
        for epoch in range(epochs):
            output = self.forward(X) # Прямое распространение - получаем предсказания
            
            self.backward(X, y, output, learning_rate) # Обратное распространение - обновляем веса
            
            # Вывод прогресса обучения каждые 10 эпох
            if epoch % 10 == 0:
                predictions = np.argmax(output, axis=1) # Получаем предсказанные классы
                accuracy = np.mean(predictions == y) # Вычисляем точность: среднее количество правильных предсказаний
                loss = np.mean((output - np.eye(10)[y]) ** 2) # Вычисляем функцию потерь (среднеквадратичная ошибка)
                print(f"Эпоха {epoch}: loss={loss:.4f}, accuracy={accuracy:.4f}") # Выводим информацию о текущей эпохе

In [26]:
# БЛОК 6: ДВУСЛОЙНАЯ НЕЙРОННАЯ СЕТЬ

class TwoLayerNN:
    """
    Класс, реализующий двуслойную нейронную сеть
    Соответствует пункту 6 задания: сеть с промежуточным слоем из 25 нейронов
    """
    
    def __init__(self, input_size, hidden_size, output_size):
        """
        Конструктор класса для двуслойной сети
        Параметры:
        input_size - количество входных признаков
        hidden_size - количество нейронов в скрытом слое (25 по заданию)
        output_size - количество нейронов в выходном слое
        """
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01 # Инициализация весов первого слоя (между входом и скрытым слоем)
        self.b1 = np.zeros((1, hidden_size)) # Инициализация смещений первого слоя
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01 # Инициализация весов второго слоя (между скрытым и выходным слоем)
        self.b2 = np.zeros((1, output_size)) # Инициализация смещений второго слоя
    
    def forward(self, X):
        """
        Прямое распространение для двуслойной сети
        """
        # Первый слой: линейная комбинация + активация ReLU
        self.z1 = np.dot(X, self.W1) + self.b1  # Линейная часть
        self.a1 = relu(self.z1)                 # Активация ReLU
        
        # Второй слой: линейная комбинация + активация сигмоида
        self.z2 = np.dot(self.a1, self.W2) + self.b2  # Линейная часть
        self.a2 = sigmoid(self.z2)                    # Активация сигмоида
        
        return self.a2 # Возвращаем выход сети
    
    def backward(self, X, y, output, learning_rate):
        """
        Обратное распространение для двуслойной сети
        """
        m = X.shape[0] # Количество примеров в пакете
        
        # Преобразование меток в one-hot encoding
        y_one_hot = np.zeros((m, 10))
        y_one_hot[np.arange(m), y] = 1
        
        # ===== ВЫЧИСЛЕНИЕ ГРАДИЕНТОВ ДЛЯ ВТОРОГО СЛОЯ =====
        
        error2 = output - y_one_hot # Ошибка на выходном слое: разница между предсказанием и истинным значением
        
        dW2 = np.dot(self.a1.T, error2) / m # Градиент для весов W2: производная по W2 = a1^T · error2
        db2 = np.sum(error2, axis=0, keepdims=True) / m # Градиент для смещений b2: производная по b2 = sum(error2)
        
        # ===== ВЫЧИСЛЕНИЕ ГРАДИЕНТОВ ДЛЯ ПЕРВОГО СЛОЯ =====
        
        # Ошибка, распространяемая обратно на первый слой:
        # error1 = error2 · W2^T * производная_ReLU(a1)
        error1 = np.dot(error2, self.W2.T) * relu_derivative(self.a1)
        
        dW1 = np.dot(X.T, error1) / m # Градиент для весов W1: производная по W1 = X^T · error1
        db1 = np.sum(error1, axis=0, keepdims=True) / m # Градиент для смещений b1: производная по b1 = sum(error1)
        
        # ===== ОБНОВЛЕНИЕ ВЕСОВ И СМЕЩЕНИЙ =====
        
        # Обновление параметров второго слоя
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        
        # Обновление параметров первого слоя
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
    
    def predict(self, X):
        """
        Функция предсказания для двуслойной сети
        """
        output = self.forward(X) # Прямое распространение
        return np.argmax(output, axis=1) # Выбор нейрона с максимальной активацией
    
    def train(self, X, y, epochs=100, learning_rate=0.1):
        """
        Функция обучения двуслойной сети
        """
        # Цикл по эпохам обучения
        for epoch in range(epochs):
            output = self.forward(X) # Прямое распространение
            
            self.backward(X, y, output, learning_rate) # Обратное распространение
            
            # Вывод прогресса обучения каждые 10 эпох
            if epoch % 10 == 0:
                predictions = np.argmax(output, axis=1) # Получаем предсказания
                accuracy = np.mean(predictions == y) # Вычисляем точность
                loss = np.mean((output - np.eye(10)[y]) ** 2) # Вычисляем функцию потерь
                print(f"Эпоха {epoch}: loss={loss:.4f}, accuracy={accuracy:.4f}") # Выводим информацию

In [27]:
# БЛОК 7: ОБУЧЕНИЕ ОДНОСЛОЙНОЙ СЕТИ

# Выводим заголовок для визуального разделения вывода
print("="*50)
print("Обучение однослойной нейронной сети")
print("="*50)

# Определяем размерность входных данных (784 пикселя для MNIST)
input_size = X_train.shape[1]  # Получаем количество столбцов = 784

# Определяем количество выходных нейронов (по одному на каждую цифру)
output_size = 10  # 10 цифр (0-9)

single_layer_nn = SingleLayerNN(input_size, output_size) # Создаем экземпляр однослойной нейронной сети

single_layer_nn.train(X_train, y_train, epochs=50, learning_rate=0.5) # Обучаем сеть: 50 эпох, скорость обучения = 0.5

Обучение однослойной нейронной сети
Эпоха 0: loss=0.2499, accuracy=0.1151
Эпоха 10: loss=0.0409, accuracy=0.8345
Эпоха 20: loss=0.0332, accuracy=0.8578
Эпоха 30: loss=0.0295, accuracy=0.8682
Эпоха 40: loss=0.0273, accuracy=0.8747


In [28]:
# БЛОК 8: ОБУЧЕНИЕ ДВУСЛОЙНОЙ СЕТИ

# Выводим заголовок для визуального разделения вывода
print("="*50)
print("Обучение двуслойной нейронной сети")
print("="*50)

# Определяем количество нейронов в скрытом слое (25 по заданию)
hidden_size = 25  # 25 нейронов в скрытом слое

two_layer_nn = TwoLayerNN(input_size, hidden_size, output_size) # Создаем экземпляр двуслойной нейронной сети

two_layer_nn.train(X_train, y_train, epochs=50, learning_rate=0.1) # Обучаем сеть: 50 эпох, скорость обучения = 0.1

Обучение двуслойной нейронной сети
Эпоха 0: loss=0.2498, accuracy=0.0530
Эпоха 10: loss=0.0928, accuracy=0.1001
Эпоха 20: loss=0.0923, accuracy=0.1116
Эпоха 30: loss=0.0918, accuracy=0.2529
Эпоха 40: loss=0.0910, accuracy=0.2840


In [29]:
# БЛОК 9: ПРЕДСКАЗАНИЯ И ОЦЕНКА ТОЧНОСТИ

# Выводим заголовок
print("="*50)
print("Оценка точности моделей")
print("="*50)

# ----- ПРЕДСКАЗАНИЯ ДЛЯ ОДНОСЛОЙНОЙ СЕТИ -----

single_layer_predictions_train = single_layer_nn.predict(X_train) # Предсказания на обучающей выборке

single_layer_predictions_test = single_layer_nn.predict(X_test) # Предсказания на тестовой выборке

# ----- ПРЕДСКАЗАНИЯ ДЛЯ ДВУСЛОЙНОЙ СЕТИ -----

two_layer_predictions_train = two_layer_nn.predict(X_train) # Предсказания на обучающей выборке

two_layer_predictions_test = two_layer_nn.predict(X_test) # Предсказания на тестовой выборке

# ----- ВЫЧИСЛЕНИЕ ТОЧНОСТИ (ACCURACY) -----

# Точность однослойной сети на обучающей выборке:
single_layer_train_acc = np.mean(single_layer_predictions_train == y_train) # Сравниваем предсказания с истинными метками, вычисляем среднее
single_layer_test_acc = np.mean(single_layer_predictions_test == y_test) # Точность однослойной сети на тестовой выборке
two_layer_train_acc = np.mean(two_layer_predictions_train == y_train) # Точность двуслойной сети на обучающей выборке
two_layer_test_acc = np.mean(two_layer_predictions_test == y_test) # Точность двуслойной сети на тестовой выборке

# ----- ВЫВОД РЕЗУЛЬТАТОВ -----

print("\nРезультаты однослойной сети:")
# :.4f означает форматирование числа с 4 знаками после запятой
print(f"Точность на обучающей выборке: {single_layer_train_acc:.4f}")
print(f"Точность на тестовой выборке: {single_layer_test_acc:.4f}")

print("\nРезультаты двуслойной сети:")
print(f"Точность на обучающей выборке: {two_layer_train_acc:.4f}")
print(f"Точность на тестовой выборке: {two_layer_test_acc:.4f}")

Оценка точности моделей

Результаты однослойной сети:
Точность на обучающей выборке: 0.8801
Точность на тестовой выборке: 0.8405

Результаты двуслойной сети:
Точность на обучающей выборке: 0.4684
Точность на тестовой выборке: 0.4735


In [30]:
# БЛОК 10: АНАЛИЗ ОШИБОК КЛАССИФИКАЦИИ

# Выводим заголовок
print("="*50)
print("Анализ ошибок классификации")
print("="*50)

# ----- НАХОЖДЕНИЕ ОШИБОЧНЫХ ПРЕДСКАЗАНИЙ -----

# np.where возвращает индексы элементов, где условие истинно
# Для однослойной сети: находим индексы, где предсказание ≠ истинной метке
errors_single = np.where(single_layer_predictions_test != y_test)[0]

# Для двуслойной сети
errors_two = np.where(two_layer_predictions_test != y_test)[0]

# ----- ВЫВОД СТАТИСТИКИ ОБ ОШИБКАХ -----

print(f"Однослойная сеть ошиблась на {len(errors_single)} из {len(y_test)} примеров")
print(f"Двуслойная сеть ошиблась на {len(errors_two)} из {len(y_test)} примеров")

# ----- ПРИМЕРЫ КОНКРЕТНЫХ ОШИБОК -----

# Если есть ошибки у однослойной сети
if len(errors_single) > 0:
    print(f"\nПример ошибки однослойной сети:")
    # Берем первую ошибку из списка
    print(f"Предсказано: {single_layer_predictions_test[errors_single[0]]}, Правильно: {y_test[errors_single[0]]}")

# Если есть ошибки у двуслойной сети
if len(errors_two) > 0:
    print(f"\nПример ошибки двуслойной сети:")
    # Берем первую ошибку из списка
    print(f"Предсказано: {two_layer_predictions_test[errors_two[0]]}, Правильно: {y_test[errors_two[0]]}")

Анализ ошибок классификации
Однослойная сеть ошиблась на 319 из 2000 примеров
Двуслойная сеть ошиблась на 1053 из 2000 примеров

Пример ошибки однослойной сети:
Предсказано: 6, Правильно: 5

Пример ошибки двуслойной сети:
Предсказано: 3, Правильно: 2


In [31]:
# БЛОК 11: ВЫВОДЫ

# Выводим заголовок
print("="*50)
print("Выводы")
print("="*50)

# Многострочный комментарий (документ-строка) с выводами о работе программы
print("""
1. Однослойная нейронная сеть с сигмоидой в качестве функции активации показала 
   базовую производительность на задаче классификации цифр.

2. Двуслойная сеть с промежуточным слоем из 25 нейронов (ReLU) и выходным слоем 
   с сигмоидой показала лучшие результаты благодаря:
   - Большей способности к обучению сложным закономерностям
   - Нелинейности, вносимой функцией ReLU

3. Обе сети успешно обучились возбуждать n-й нейрон при классификации цифры n.

4. Двуслойная сеть демонстрирует более высокую точность как на обучающей, 
   так и на тестовой выборке, что свидетельствует о ее лучшей способности 
   к обобщению.

5. Для дальнейшего улучшения результатов можно:
   - Увеличить количество эпох обучения
   - Настроить скорость обучения
   - Добавить больше слоев и нейронов
   - Применить методы регуляризации
""")

Выводы

1. Однослойная нейронная сеть с сигмоидой в качестве функции активации показала 
   базовую производительность на задаче классификации цифр.

2. Двуслойная сеть с промежуточным слоем из 25 нейронов (ReLU) и выходным слоем 
   с сигмоидой показала лучшие результаты благодаря:
   - Большей способности к обучению сложным закономерностям
   - Нелинейности, вносимой функцией ReLU

3. Обе сети успешно обучились возбуждать n-й нейрон при классификации цифры n.

4. Двуслойная сеть демонстрирует более высокую точность как на обучающей, 
   так и на тестовой выборке, что свидетельствует о ее лучшей способности 
   к обобщению.

5. Для дальнейшего улучшения результатов можно:
   - Увеличить количество эпох обучения
   - Настроить скорость обучения
   - Добавить больше слоев и нейронов
   - Применить методы регуляризации

